In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import seaborn as sns
from scipy import stats
from statsmodels.stats import power

dpi=400
default_hatch_patterns = ["o" , "x", "." , "*" , "." , "\\" , "O", ".", "*"]
default_color = "0.80"
default_box_width = 14
default_box_marker = "|"
default_box_marker_size = 20
default_box_marker_alpha = 0.8
default_grid_color = "0.8"
df = pd.read_csv("TYU07_corrigido.csv").drop('Unnamed: 0', axis=1)

In [ ]:
def draw_t_score(t_sample, alpha, dfreedom, direction, title):
    
    if direction == "<":
        tc = stats.t.ppf(alpha, dfreedom)
    elif direction == ">":
        tc = stats.t.ppf(1 - alpha, dfreedom)
    else:
        alpha = alpha/2
        tc = stats.t.ppf(1 - alpha, dfreedom)
    
    lim = math.ceil(max(3, abs(t_sample) + 1))
    x = np.arange(-lim, lim, 0.001)
    y = stats.t.pdf(x, dfreedom) #stats.norm.pdf(x, 0, 1)
    if direction == "<":
        t = x[x < tc]
    elif direction == ">":
        t = x[x > tc]
    else:
        t = x[x < -tc]
        t2 = x[x > tc]
        
    fig, ax = plt.subplots(figsize=(7,4))    
    ax.plot(x, y, color='black', lw=0.8)
    ax.fill_between(t, 0, stats.t.pdf(t, dfreedom), facecolor='0.7')
    if direction != "<" and direction != ">":
        ax.fill_between(t2, 0, stats.t.pdf(t2, dfreedom), facecolor='0.7')        

    ax.set(xbound =(-lim, lim), ybound = (0,0.5))
    
    # Insere anotação sobre a linha de t do sample
    ax.annotate('t_sample = {:0.3f}'.format(t_sample), xy=(t_sample, 0), xytext=(0, 25),
            xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='bottom',
            arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))
    
    # Insere anotação sobre a linha de t critico
    ax.annotate('t_c = ' + str(round(tc,3)) + '\nα = ' + str(alpha), xy=(tc, 0), xytext=(0, -25),
            xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='top',
            arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))    
    
    if direction != "<" and direction != ">":
        # Insere anotação sobre a linha de t critico
        ax.annotate('-t_c = ' + str(round(-tc,3)) + '\nα = ' + str(alpha), xy=(-tc, 0), xytext=(0, -25),
                xycoords=('data', 'axes fraction'), textcoords='offset points', horizontalalignment="center", verticalalignment='top',
                arrowprops=dict(arrowstyle='-|>', fc='black', shrinkA=0, shrinkB=0), bbox=dict(boxstyle="round", alpha=0.7, fc='white'))    
    

# Segunda parte – Análise da variável Nota no ENEM

## Questão 2
Há uma grande preocupação em estimar o comportamento da variável Nota no ENEM dos alunos, de maneira a caracterizar melhor seu o perfil. Alguém da universidade sugeriu que você retirasse uma amostra de 25 alunos e registrasse os valores de Nota no ENEM3. Considere que a variância populacional de Nota no ENEM é DESCONHECIDA. Com base nos resultados encontrados responda os itens a seguir (lembre-se de excluir inicialmente as linhas com dados perdidos).

### 2.a) Construa um gráfico de probabilidade normal para os valores da amostra.

In [ ]:
# Define os dados para trabalhar na parte 2
N = 25
dfreedom = N - 1

# Garante que sempre serão gerados os mesmos números aleatórios
np.random.seed(0)

# Retira 25 samples da nota enem
sampleENEM = np.random.choice(df["Nota ENEM"].dropna().tolist(), N)

# Salva os samples em um .csv com ajuda do pandas dataframe
pd.DataFrame(sampleENEM, columns=["Amostras da nota ENEM"]).to_csv("Dados\Q2_samples.csv", encoding='utf-8', index=False)

sample_mean = np.mean(sampleENEM)
sample_std = np.std(sampleENEM)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plot = stats.probplot(sampleENEM, plot=ax)
ax.set(title="Gráfico de probabilidade normal da amostra da variável ENEM")
ax.set(xlabel="Quantis", ylabel="Nota")
ax.grid(color=default_grid_color)

# Seleciona os pontos
ax.get_lines()[0].set(color = 'black', markersize=10)
# Seleciona a linha de tendencia
ax.get_lines()[1].set(color = 'black', ls = 'dashed')

# Insere informação adicional (caixa)
ax.text(-1.5, 650, "Assimetria: " + str(round(stats.skew(sampleENEM),3)) + "\nCurtose: " + str(round(stats.kurtosis(sampleENEM),3)), va="center", ha="left",
        bbox=dict(boxstyle="round", alpha=1, fc='white'))

# Salva figura na pasta dedicada
plt.savefig('Dados\Q2a.png', bbox_inches='tight', dpi=dpi)

### 2.a.1 - É possível considerar que os dados provêm de uma população com distribuição normal? JUSTIFIQUE.

Com base no grafico anterior ou analisando os dados da populaçao?

### 2.a.2 – Com base na resposta da letra a.1 você recomendaria a utilização de técnicas como Intervalo de confiança e teste paramétrico de média? JUSTIFIQUE.

Ver depois (utilizar teste t)

### 2.b) Independente dos resultados da letra a), encontre o intervalo de 95% de confiança para a média populacional. Interprete o resultado.

In [ ]:
# Encontra os intervalos de confiança t para alpha=0.05 e seleciona o positivo
tc = stats.t.ppf(q = 1-alpha/2, df = dfreedom)

# Calcula a margem de erro a partir do valor t de 95% de confiança
tc = stats.t.interval(0.95, dfreedom)[1] # Calcula o t crítico para 95% de confiança
error_95 = tc * sample_std / math.sqrt(N) # Calcula o erro para 95% de confiança
lim_inf = sample_mean - error_95
mean = sample_mean
lim_sup = sample_mean + error_95
print("Para o indíce de confiança de 95%, foi encontrado um erro de {0:.2f} sobre a média amostral {1:.2f}, produzindo um intervalo de {2:.2f} a {3:.2f} com tc = {4:.3f}".format(
        error_95, mean, lim_inf, lim_sup, tc))

### 2.c) Independente dos resultados da letra a), qual seria o tamanho mínimo de amostra necessário para obter um intervalo de 95% de confiança para a média populacional da Nota no ENEM, com uma precisão de 5 pontos? A amostra coletada é suficiente? JUSTIFIQUE.

In [ ]:
# Encontra os intervalos de confiança t para alpha=0.05 e seleciona o positivo
tc = stats.t.interval(0.95, dfreedom)[1] # Calcula o t crítico para 5% de significância
N_pop = df["Nota ENEM"].count() # Calcula o tamanho da população
n_zero = ((tc * sample_std) / 5)**2 # Cálculo do tamanho mínimo de amostra
n_corr = (N_pop * n_zero) / (N_pop + n_zero) # Correção de tamanho de amostra
print("O tamanho mínimo de intervalo é de " + str(math.ceil(n_corr)) + " amostras.")

### d) Independente dos resultados da letra a), a direção da TYU acredita que a média de Nota no ENEM dos alunos é maior do que 450 pontos, por pesquisas anteriores. Aplicando o teste estatístico apropriado, os dados confirmam isso, a 5% de significância? JUSTIFIQUE.

Vamos utilizar o teste t (justificar)

H0: Nota ENEM = 450

H1: Nota ENEM >  450

Alpha = 5% (cauda direita)

In [ ]:
alpha = 0.05
hyp_mean = 450
tc = stats.t.ppf(1-alpha, dfreedom) # Calcula t crítico para 5 de significancia
t = (sample_mean - hyp_mean)/(sample_std/math.sqrt(dfreedom)) # Calcula valor t da amostra
p = 1 - stats.t.cdf(t, df=dfreedom) # Calcula o valor-p

In [ ]:
print("Como o valor p {0} é menor que o índice de significância (alpha) para esta situação ({1}), a suposição H0 de que a nota do enem não é superior a 450 é rejeitada. Portanto, é possível dizer com confiança de 95% que a média da nota do ENEM é superior a 450 pontos.".format(round(p,4),round(alpha,4)))

In [ ]:
draw_t_score(t_sample = t, alpha=alpha, dfreedom = dfreedom, direction = '>', title = 't')

# Salva figura na pasta dedicada
plt.savefig('Dados\Q2d.png', bbox_inches='tight', dpi=dpi)

### e) Analise o intervalo de confiança da letra b). Os limites encontrados corroboram os resultados encontrados na letra d)? JUSTIFIQUE.

In [ ]:
print("Sim, como há 95% de certeza que a média se encontra entre {0} e {1}, a afirmação de que a média está acima de 450 pontos faz sentido também.".format(round(interval_95[0],2), round(interval_95[2],2)))

### f) Independente dos resultados da letra a), suponha 5% de significância e o desvio padrão amostral como boa estimativa do desvio padrão populacional. Se a média real da Nota no ENEM fosse de 470 pontos, qual seria o poder do teste? Você acha o valor aceitável? JUSTIFIQUE.

H0: Nota ENEM <= 450

H1: Nota ENEM > 450

In [ ]:
alpha = 0.05
hyp_mean = 450
pop_mean = 470
pop_std = sample_std
dist_std = (pop_mean-hyp_mean)/pop_std
poder = power.tt_solve_power(effect_size=abs(dist_std), nobs=N, alpha=alpha, alternative='larger')
print("Distância entre média real e média testada: 20\nDesvio padrão populacional estimado: {0:.2f}\nDistância em desvio padrão: {1:.4f}".format(pop_std, dist_std))
print("Poder do teste: " + str(round(poder, 3)))

### g) Independente dos resultados da letra a), qual deveria ser o tamanho mínimo de amostra para detectar com 95% de probabilidade que a média da Nota no ENEM dos alunos é igual a 470 pontos. Suponha 5% de significância e o desvio padrão amostral como boa estimativa do desvio padrão populacional. A amostra coletada é suficiente? JUSTIFIQUE.

In [ ]:
# n_min = power.tt_solve_power(effect_size=abs(dist_std), alpha=0.05, power=0.95)
# print("Para detectar com 95% de probabilidade que a média da nota ENEM é 470, significa rejeitar a hipótese de que a média é 450 com 95% de probabilidade quando a média real é 470")
# print("Número mínimo de amostras: " + str(math.ceil(n_min)))